In [82]:
import pandas as pd
import dateutil.parser
import time
import functools
import collections
import itertools

In [83]:
data = pd.read_csv(".data/stock_data.csv").drop(columns=["Unnamed: 0"])
#data = data.reset_index()
data["Date"] = data["Date"].apply(lambda x: dateutil.parser.parse(x))
data["Date"] = data["Date"].dt.date
data['week'] = data['Date'].apply(lambda x: x.isocalendar()[1])
data['year'] = data['Date'].apply(lambda x: x.year)
data['day_of_week'] = data['Date'].apply(lambda x: x.isocalendar()[2])
data = data.set_index("Date")
data

,Open,High,Low,Close,Volume,Dividends,Stock Splits,week,year,day_of_week
Date,,,,,,,,,,
2019-02-14,40.844916,41.217959,40.765492,41.107250,87342800,0.00,0.0,7,2019,4
2019-02-15,41.215546,41.323849,40.854534,41.015785,98507200,0.00,0.0,7,2019,5
2019-02-19,40.844911,41.261277,40.791962,41.138531,75891200,0.00,0.0,8,2019,2
2019-02-20,41.201105,41.713743,41.152971,41.403271,104457600,0.00,0.0,8,2019,3
2019-02-21,41.347916,41.485098,40.986904,41.169815,68998800,0.00,0.0,8,2019,4
...,...,...,...,...,...,...,...,...,...,...
2024-02-07,190.397053,190.806534,188.369641,189.168625,53439000,0.00,0.0,6,2024,3
2024-02-08,189.148646,189.298448,187.111252,188.080017,40962000,0.00,0.0,6,2024,4
2024-02-09,188.649994,189.990005,188.000000,188.850006,45099900,0.24,0.0,6,2024,5


In [84]:
daily_open_close = data.groupby(by=["year","week","day_of_week"])[["Open","Close"]].mean()
open_close = daily_open_close.reset_index("day_of_week")
grouped = open_close.groupby(by=["year","week"])
open_close["start_of_week"] = grouped[["day_of_week"]].min()
open_close["end_of_week"] = grouped[["day_of_week"]].max()
open_close.sort_values(by = ["year","week","day_of_week"])

day_of_week        Open       Close  start_of_week  end_of_week
year week                                                                 
2019 1               1   70.410440   70.911530              1            2
     1               2   70.524783   71.429665              1            2
     7               4   40.844916   41.107250              4            5
     7               5   41.215546   41.015785              4            5
     8               2   40.844911   41.138531              2            5
...                ...         ...         ...            ...          ...
2024 6               3  190.397053  189.168625              1            5
     6               4  189.148646  188.080017              1            5
     6               5  188.649994  188.850006              1            5
     7               1  188.419998  187.149994              1            2
     7               2  185.770004  185.039993              1            2

[1258 rows x 5 columns]

In [85]:
unique_weeks = open_close.reset_index()[["year","week","start_of_week","end_of_week"]].drop_duplicates()
weekly_opens = unique_weeks.rename(columns = {"start_of_week":"day_of_week"}).set_index(["year","week","day_of_week"]).join(daily_open_close["Open"]).drop(columns = ["end_of_week"]).reset_index().set_index(["year","week"]).drop(columns="day_of_week")
weekly_closes = unique_weeks.rename(columns = {"end_of_week":"day_of_week"}).set_index(["year","week","day_of_week"]).join(daily_open_close["Close"]).drop(columns = ["start_of_week"]).reset_index().set_index(["year","week"]).drop(columns="day_of_week")
unique_weeks = unique_weeks.set_index(["year","week"])
unique_weeks["Open"] = weekly_opens
unique_weeks["Close"] = weekly_closes
unique_weeks.drop(columns=["start_of_week","end_of_week"], inplace=True)
unique_weeks.to_csv(".data/cleaned_stock_data.csv")

In [86]:
news = pd.read_csv(".data/processed_news.csv")
news = news.drop(columns=["Unnamed: 0"])
news

,title,description,published date,url,publisher,fetched_title,fetched_body,week,year,gpt_results
0,New York Times (NYT) Cuts Stories for Apple (A...,New York Times (NYT) Cuts Stories for Apple (A...,2019-03-18 00:00:00,https://news.google.com/rss/articles/CBMicmh0d...,"{'href': 'https://www.bloomberg.com', 'title':...",Are you a robot?,Why did this happen?\n\nPlease make sure your ...,12,2019,Summary: The news discusses a recent event tha...
1,"Apple lands the WSJ for its forthcoming ""Netfl...","Apple lands the WSJ for its forthcoming ""Netfl...",2019-03-22 00:00:00,https://news.google.com/rss/articles/CBMiWWh0d...,"{'href': 'https://www.businessinsider.com', 't...",Apple's news service just landed The Wall Stre...,This is an excerpt from a story delivered excl...,12,2019,Summary: Apple has convinced The Wall Street J...
2,The Biggest Threats Facing Tech Stocks Now - I...,The Biggest Threats Facing Tech Stocks Now In...,2019-03-22 00:00:00,https://news.google.com/rss/articles/CBMiT2h0d...,"{'href': 'https://www.investopedia.com', 'titl...",The Biggest Threats Facing Tech Stocks Now,It would appear that the good old days are bac...,12,2019,"Summary: Tech stocks, including Apple Inc. (AA..."
3,Apple (AAPL) Driverless Cars Need More Human I...,Apple (AAPL) Driverless Cars Need More Human I...,2019-02-13 00:00:00,https://news.google.com/rss/articles/CBMicGh0d...,"{'href': 'https://www.bloomberg.com', 'title':...",Are you a robot?,Why did this happen?\n\nPlease make sure your ...,7,2019,Summary: The news discusses the reasons behind...
4,Apple and Fitbit are leaders in the growing sm...,Apple and Fitbit are leaders in the growing sm...,2019-02-13 00:00:00,https://news.google.com/rss/articles/CBMiW2h0d...,"{'href': 'https://news.alphastreet.com', 'titl...",Apple and Fitbit are leaders in the growing sm...,The US smartwatch market has grown significant...,7,2019,Summary: The US smartwatch market has seen sig...
...,...,...,...,...,...,...,...,...,...,...
888,Here's How Not Holding Apple (AAPL) Affected M...,Here's How Not Holding Apple (AAPL) Affected M...,2024-02-13 08:00:00+00:00,https://news.google.com/rss/articles/CBMiTWh0d...,"{'href': 'https://finance.yahoo.com', 'title':...",Here’s How Not Holding Apple (AAPL) Affected M...,"Mairs & Power, an investment advisor, released...",7,2024,Summary: Mairs & Power Growth Fund outperforme...
889,AAPL Stock Weakness Sets Up Possible Bear Call...,AAPL Stock Weakness Sets Up Possible Bear Call...,2024-02-13 08:00:00+00:00,https://news.google.com/rss/articles/CBMiUGh0d...,"{'href': 'https://www.investors.com', 'title':...",AAPL Stock Weakness Sets Up Possible Bear Call...,Apple (AAPL) has been underperforming versus t...,7,2024,Summary: Apple (AAPL) stock has been underperf...
890,Apple's (AAPL) Longest-Serving Designer Bart A...,Apple's (AAPL) Longest-Serving Designer Bart A...,2024-02-13 08:00:00+00:00,https://news.google.com/rss/articles/CBMidGh0d...,"{'href': 'https://www.bloomberg.com', 'title':...",Are you a robot?,Why did this happen?\n\nPlease make sure your ...,7,2024,Summary: The news discusses a possible issue w...
891,Apple (NASDAQ:AAPL) to Settle Chip Theft Lawsu...,Apple (NASDAQ:AAPL) to Settle Chip Theft Lawsu...,2024-02-12 08:00:00+00:00,https://news.google.com/rss/articles/CBMib2h0d...,"{'href': 'https://www.tipranks.com', 'title': ...",Apple (NASDAQ:AAPL) to Settle Chip Theft Lawsu...,Tech giant Apple (NASDAQ:AAPL) is planning to ...,7,2024,Summary: Apple is planning to settle a lawsuit...


In [87]:
joined_data = unique_weeks.join(news.groupby(by=["year","week"])[["gpt_results"]].sum())

In [88]:
joined_data

Open       Close  \
year week                           
2019 1      70.410440   71.429665   
     7      40.844916   41.015785   
     8      40.844911   41.629505   
     9      41.915913   42.110847   
     10     42.284134   41.615067   
...               ...         ...   
2024 3     181.927853  191.315872   
     4     192.054929  192.174774   
     5     191.765299  185.613159   
     6     187.910213  188.850006   
     7     188.419998  185.039993   

                                                 gpt_results  
year week                                                     
2019 1                                                   NaN  
     7     Summary: The news discusses the reasons behind...  
     8     Summary: Apple is preparing to launch upgraded...  
     9     Summary: Fitbit, Inc. shares fell despite post...  
     10    Summary: The news discusses a recent event rel...  
...                                                      ...  
2024 3     Summary: Tsai Capital Corporation released its...  
     4     Summary: Morningstar provides insights on what...  
     5     Summary: Apple reported fiscal first-quarter e...  
     6     Summary: Apple stock struggling to reach new h...  
     7     Summary: Mairs & Power Growth Fund outperforme...  

[263 rows x 3 columns]

In [89]:
print(joined_data.iloc[1]["gpt_results"])

Summary: The news discusses the reasons behind a recent event and suggests reviewing the Terms of Service and Cookie Policy for more information.

Keywords: Apple stock analyst, news data, noisy, relevant, keywords, Terms of Service, Cookie Policy.Summary: The US smartwatch market has seen significant growth in the past year, with Apple and Fitbit dominating sales. New devices like the Apple Watch Series 4 are expected to increase penetration among older demographics. Other companies like Garmin and Fossil are entering the smartwatch market to increase their market share. Apple's Wearables segment saw revenue growth of 33% in its most recent quarter, while Fitbit's smartwatch revenue grew to 49% of total revenue.

Keywords: Apple, Fitbit, smartwatch market, US, growth, NPD report, Apple Watch Series 4, Garmin, Fossil, revenue, Wearables segment, market share, earnings results, stock price.Summary: The article discusses how defining a business flop for a company like Apple can be differ

In [95]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import langchain
gpt = ChatOpenAI(model="gpt-4")
company_profile = """Description: Apple Inc. (AAPL) is an American multinational technology company that specializes in consumer electronics, software, and services. It is best known for its iPhone smartphones, iPad tablets, Mac computers, and services like the App Store, Apple Music, and iCloud. Apple's emphasis on design and innovation has made it one of the most valuable companies in the world.

Positive Factors:
- **Innovation and Product Launches**: Continuous innovation and the release of new products can drive consumer interest and sales.
- **Brand Loyalty**: Strong brand recognition and a loyal customer base can sustain revenue and profit margins.
- **Services Growth**: Expansion of Apple's services sector, including streaming, financial services, and cloud storage, diversifies revenue streams.
- **Global Expansion**: Entry into emerging markets can provide new revenue sources.
- **Macroeconomic Trends**: A strong economic environment, with high consumer spending and low unemployment rates, can increase demand for Apple's products.

Negative Factors:
- **Competition**: Intense competition from other tech giants and emerging players can impact market share and pricing.
- **Supply Chain Disruptions**: Reliance on global supply chains makes Apple vulnerable to disruptions, tariffs, and geopolitical tensions.
- **Regulatory Challenges**: Increased scrutiny and regulation in key markets, including antitrust investigations and privacy regulations, can affect operations and costs.
- **Market Saturation**: High penetration rates in key markets can slow growth, making it challenging to maintain high sales volumes.
- **Macroeconomic Downturns**: Economic downturns, high inflation rates, or rising interest rates can reduce consumer spending on non-essential goods, impacting sales."""

news_summary_prompt = ChatPromptTemplate.from_template(f"""Instruction: Forecast next week stock return (price change) for symbol, given the company profile, historical weekly news summary, keywords, and stock returns, and optionally the examples from other stocks of a similar company. Return a single stock return.
The trend is represented by bins "D5+", "D5", "D4", "D3", "D2", "D1", "U1", "U2", "U3", "U4", "U5", "U5+", where "D5+" means price dropping more than 5%, D5 means price dropping between 4% and 5%, "D4" means price dropping between 3% and 4%, "U5+" means price rising more than 5%, "U5" means price rising between 4% and 5%, "D4" means price rising between 3% and 4%, etc. Company Profile: {company_profile}
Recent News: News are ordered from oldest news to latest news. {{news}}""")


news_summary_chain = news_summary_prompt | gpt | StrOutputParser()
get_result = lambda p: news_summary_chain.invoke(p)
gpt_results = get_result({"news": joined_data["gpt_results"].iloc[-1]})
print(gpt_results)

Based on the company profile, current news, and market trends, the forecast for Apple Inc.’s stock return for next week falls within the "U1" bin. This indicates that the price is expected to rise between 0% and 1%. This is due to the positive factors such as strong brand loyalty and innovative product launches. However, potential challenges such as intense competition and supply chain disruptions could limit the stock's growth. The recent news of a lawsuit settlement and analysts' forecast for a 9.95% increase over the next twelve months contribute to this prediction. Please note, this is a forecast and actual results may vary. Always consider multiple factors and consult with a financial advisor before making investment decisions.


In [91]:
joined_data = joined_data[~joined_data["gpt_results"].isna()]

In [92]:
def with_n_history(iterable, n):
    "Returns the data along with a history of n records from the iterator"
    # with_n_history('ABCDEFG', 4) --> A AB ABC ABCD BCDE CDEF DEFG
    it = iter(iterable)
    window = collections.deque(maxlen=n)
    for x in it:
        window.append(x)
        yield tuple(window)
def batched(iter, batch_size):
    temp = []
    for i in iter:
        temp.append(i)
        if len(temp) == batch_size:
            yield temp
            temp = []
    yield temp

In [94]:
res = []
for batch in batched(with_n_history(joined_data["gpt_results"], 8), 10):
    print("Starting Batch")
    def create_summary_section(summaries):
        sum_length = len(summaries)
        out = ""
        for idx, summary in enumerate(summaries):
            out += f"News from {sum_length - idx} weeks ago: {summary}\n"
        return out
    morphed_batch = [{"news": create_summary_section(summaries)} for summaries in batch]
    res.extend(news_summary_chain.batch(morphed_batch))
    time.sleep(65)
res

Starting Batch


['Based on the positive factors such as continuous innovation and product launches, strong brand loyalty, services growth, and global expansion, as well as the recent news about revenue growth in Apple\'s Wearables segment, it is likely that Apple\'s stock will have a positive return next week. Considering the historical performance of Apple\'s stock, which has generally been positive in response to such factors, I would forecast a stock return in the "U2" to "U4" range, indicating a price increase between 2% to 4%.',
 'Based on the recent news and factors affecting Apple Inc. (AAPL), it is forecasted that the stock return for next week will be in the "D3" bin, indicating a price drop between 2% and 3%. The challenges with declining iPhone sales, particularly in Greater China, along with the regulatory challenges and competitive landscape, are likely to impact the stock price negatively. Additionally, the recent insider trading scandal involving Apple\'s former senior director may also

In [ ]:
for i in joined_data["gpt_results"]:
    print(i)
    break

nan


In [ ]:
res

['Based on the positive factors such as innovation, brand loyalty, and services growth, as well as recent news indicating revenue growth in the Wearables segment, it is likely that Apple Inc. (AAPL) will experience a positive stock return next week. Considering the historical trends and the overall positive company profile, the forecasted stock return for AAPL next week is "U3" (price rising between 2% and 3%).',
 'Based on the positive factors such as innovation, brand loyalty, and services growth, as well as the upcoming launch of upgraded AirPods with new features, it is likely that Apple\'s stock price will experience a positive trend next week. Additionally, the negative factors like competition and supply chain disruptions may have a limited impact given the overall strength of the company. Therefore, the forecasted stock return for Apple (AAPL) next week is in the "U1" to "U3" range, indicating a modest to moderate price increase.',
 'Based on the recent news and competition bet

In [ ]:
chars = ["D5+", "D5", "D4", "D3", "D2", "D1", "U1", "U2", "U3", "U4", "U5", "U5+"]
limits = [float("-inf"), -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, float("inf")]
def get_char(haystack, char_set):
    for char in char_set:
        if haystack.find(char) != -1:
            return char
    return None
def map_decimal_to_bin(decimal, bins, limits):
    bins = ["D5+", "D5", "D4", "D3", "D2", "D1", "U1", "U2", "U3", "U4", "U5", "U5+"]
    # Find the bin
    for i in range(len(limits) - 1):
        if limits[i] <= decimal < limits[i + 1]:
            return bins[i]
    return "Error: Decimal number does not fit any bin"

res_token = [get_char(response, chars) for response in res]
res_token
joined_data["gpt_analysis"] = res
joined_data["predicted_token"] = res_token
joined_data

/var/folders/b2/kj73pqdx4sxdntx6q_0yhbqh0000gp/T/ipykernel_64756/4101961673.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  joined_data["gpt_analysis"] = res
/var/folders/b2/kj73pqdx4sxdntx6q_0yhbqh0000gp/T/ipykernel_64756/4101961673.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  joined_data["predicted_token"] = res_token


Open       Close  \
year week                           
2019 7      40.844916   41.015785   
     8      40.844911   41.629505   
     9      41.915913   42.110847   
     10     42.284134   41.615067   
     11     42.236000   44.794376   
...               ...         ...   
2024 3     181.927853  191.315872   
     4     192.054929  192.174774   
     5     191.765299  185.613159   
     6     187.910213  188.850006   
     7     188.419998  185.039993   

                                                 gpt_results  \
year week                                                      
2019 7     Summary: The news discusses the reasons behind...   
     8     Summary: Apple is preparing to launch upgraded...   
     9     Summary: Fitbit, Inc. shares fell despite post...   
     10    Summary: The news discusses a recent event rel...   
     11    Summary: President Trump defends calling Apple...   
...                                                      ...   
2024 3     Summary: Tsai Capital Corporation released its...   
     4     Summary: Morningstar provides insights on what...   
     5     Summary: Apple reported fiscal first-quarter e...   
     6     Summary: Apple stock struggling to reach new h...   
     7     Summary: Mairs & Power Growth Fund outperforme...   

                                                gpt_analysis predicted_token  \
year week                                                                      
2019 7     Based on the positive factors such as innovati...              U3   
     8     Based on the positive factors such as innovati...              U1   
     9     Based on the recent news and competition betwe...              U3   
     10    Based on the company profile, positive factors...              U1   
     11    Based on the company profile, recent news, and...              U2   
...                                                      ...             ...   
2024 3     Based on the recent news and analyst upgrades,...              U1   
     4     Based on the company profile, recent news, and...              U4   
     5     Based on the company profile, recent news, and...              D2   
     6     Based on the positive factors such as innovati...              U3   
     7     Based on the company profile, recent news, and...            None   

           real_performance real_token  
year week                               
2019 7             0.416595       None  
     8             1.884707       None  
     9             0.462908       None  
     10           -1.607754       None  
     11            5.711378         U5  
...                     ...        ...  
2024 3             4.907078         U5  
     4             0.062363       None  
     5            -3.314495       None  
     6             0.497640       None  
     7            -1.826635       None  

[260 rows x 7 columns]

In [ ]:
joined_data["real_performance"] = 100*(joined_data["Close"] - joined_data["Open"])/joined_data["Close"]
joined_data

/var/folders/b2/kj73pqdx4sxdntx6q_0yhbqh0000gp/T/ipykernel_64756/2163904166.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  joined_data["real_performance"] = 100*(joined_data["Close"] - joined_data["Open"])/joined_data["Close"]


Open       Close  \
year week                           
2019 7      40.844916   41.015785   
     8      40.844911   41.629505   
     9      41.915913   42.110847   
     10     42.284134   41.615067   
     11     42.236000   44.794376   
...               ...         ...   
2024 3     181.927853  191.315872   
     4     192.054929  192.174774   
     5     191.765299  185.613159   
     6     187.910213  188.850006   
     7     188.419998  185.039993   

                                                 gpt_results  \
year week                                                      
2019 7     Summary: The news discusses the reasons behind...   
     8     Summary: Apple is preparing to launch upgraded...   
     9     Summary: Fitbit, Inc. shares fell despite post...   
     10    Summary: The news discusses a recent event rel...   
     11    Summary: President Trump defends calling Apple...   
...                                                      ...   
2024 3     Summary: Tsai Capital Corporation released its...   
     4     Summary: Morningstar provides insights on what...   
     5     Summary: Apple reported fiscal first-quarter e...   
     6     Summary: Apple stock struggling to reach new h...   
     7     Summary: Mairs & Power Growth Fund outperforme...   

                                                gpt_analysis predicted_token  \
year week                                                                      
2019 7     Based on the positive factors such as innovati...              U3   
     8     Based on the positive factors such as innovati...              U1   
     9     Based on the recent news and competition betwe...              U3   
     10    Based on the company profile, positive factors...              U1   
     11    Based on the company profile, recent news, and...              U2   
...                                                      ...             ...   
2024 3     Based on the recent news and analyst upgrades,...              U1   
     4     Based on the company profile, recent news, and...              U4   
     5     Based on the company profile, recent news, and...              D2   
     6     Based on the positive factors such as innovati...              U3   
     7     Based on the company profile, recent news, and...            None   

           real_performance real_token  
year week                               
2019 7             0.416595       None  
     8             1.884707       None  
     9             0.462908       None  
     10           -1.607754       None  
     11            5.711378         U5  
...                     ...        ...  
2024 3             4.907078         U5  
     4             0.062363       None  
     5            -3.314495       None  
     6             0.497640       None  
     7            -1.826635       None  

[260 rows x 7 columns]

In [ ]:
map_decimal_to_bin(3.9, chars, limits)

'U4'

In [ ]:
joined_data["real_token"] = joined_data["real_performance"].apply(functools.partial(map_decimal_to_bin, bins = chars, limits = limits))

/var/folders/b2/kj73pqdx4sxdntx6q_0yhbqh0000gp/T/ipykernel_64756/1460066568.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  joined_data["real_token"] = joined_data["real_performance"].apply(functools.partial(map_decimal_to_bin, bins = chars, limits = limits))


In [ ]:
import numpy as np

In [ ]:
np.unique((joined_data["real_token"] == joined_data["predicted_token"]), return_counts=True)

(array([False,  True]), array([231,  29]))